<a href="https://colab.research.google.com/github/manikanta-eng/HPC/blob/main/assignment_6_hpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task 1 : Identification of NUMA Architecture and System
Topology**

In [1]:
!lscpu  | grep NUMA

NUMA node(s):                            1
NUMA node0 CPU(s):                       0,1


In [4]:
!apt-get update
!apt-get install -y numactl


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
numactl is already the newest version (2.0.14-3ubuntu2).
0 upgraded, 0 newly installed, 0 to r

In [5]:
!numactl --hardware

available: 1 nodes (0)
node 0 cpus: 0 1
node 0 size: 12975 MB
node 0 free: 8631 MB
node distances:
node   0 
  0:  10 


In [6]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

**Task 2 Memory Bandwidth Measurement on Single NUMA Node**

In [7]:
import numpy as np
import time
N = 100_000_000
a = np.ones(N, dtype=np.float64)
b = np.ones(N, dtype=np.float64)
start = time.time()
c = a + b
end = time.time()
execution_time = end - start
bytes_accessed = 3 * a.nbytes
bandwidth = bytes_accessed / execution_time / 1e9
print("Execution Time:", execution_time, "seconds")
print("Memory Bandwidth:", bandwidth, "GB/s")


Execution Time: 0.5081963539123535 seconds
Memory Bandwidth: 4.722584059337659 GB/s


In [14]:
!numactl --cpunodebind=0 --membind=0 python3 benchmark.py

Execution Time: 0.4432039260864258 seconds
Memory Bandwidth: 5.415114485091439 GB/s


**task 3 Local vs Remote Memory Access Performance (NUMA-
Aware Execution)**

In [17]:
import numpy as np
import time

N = 100_000_000


a = np.ones(N, dtype=np.float64)
b = np.ones(N, dtype=np.float64)

start = time.time()


c = a + b
end = time.time()

execution_time = end - start


bytes_accessed = 3 * a.nbytes

bandwidth = bytes_accessed / execution_time / 1e9

print("Execution Time:", execution_time, "seconds")
print("Memory Bandwidth:", bandwidth, "GB/s")


Execution Time: 2.2966105937957764 seconds
Memory Bandwidth: 1.045018257114866 GB/s


**task 4 Impact of Multi-threaded Memory Access on NUMA
Systems**

In [18]:
import numpy as np
import threading
import time

N = 50_000_000
a = np.ones(N)

def worker():
    global a
    a = a * 2

threads = []

start = time.time()

for i in range(4):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

end = time.time()

print("Execution Time:", end - start, "seconds")


Execution Time: 1.130096673965454 seconds


**task 5 Title  Comparative Study of NUMA-Unaware vs NUMA-Aware
Execution**

In [ ]:
import numpy as np
import threading
import time

N = 50_000_000
a = np.ones(N)

def worker():
    global a
    a = a * 2

threads = []

start = time.time()

for i in range(4):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

end = time.time()

print("Execution Time:", end - start, "seconds")


In [16]:
!numactl --cpunodebind=0 --membind=0 python3 task5.py

Execution Time: 0.9741628170013428 seconds
